# Transition Metal Halides

In [1]:
from ase import Atom, Atoms
from ase.build import mx2, molecule
from ase.data import chemical_symbols
from ase.io import read, write
from ase.spacegroup import get_spacegroup
from ase.visualize import view
from itertools import combinations
import numpy as np
import random

class TrandionMetalHalidesBuilder:
    """
    用于搭建高对称二维过渡金属卤化物结构的 python 类。
    """
    def __init__(self, prototype="MX3", m_element="Cr", x_element="I", repeating_unit="1", 
                 lattice_a=0.0, lattice_b=0.0, lattice_c=0.0, supercell_matrix=(3, 3, 1), vacuum=15.0, n_layers=1, layer_distance=5.0, **kwargs):
        self.prototype = prototype
        self.m_element = m_element
        self.x_element = x_element
        self.repeating_unit = repeating_unit
        self.lattice_a = lattice_a
        self.lattice_b = lattice_b
        self.lattice_c = lattice_c
        self.supercell_matrix = supercell_matrix
        self.vacuum = vacuum
        self.n_layers = n_layers
        self.layer_distance = layer_distance
        
        if self.supercell_matrix[2] != 1:
            print(f"Warning: The third component of supercell_matrix should be 1, but got {self.supercell_matrix[2]}. It will be set to 1.")
            self.supercell_matrix = (self.supercell_matrix[0], self.supercell_matrix[1], 1)
        
    def predefined_systems(self):
        predefined_dict = {
            "MX3": {
                "symbols": [self.m_element] * 6 + [self.x_element] * 18,
                "cell": [6.96855, 6.96855, 20.75586, 90, 90, 120],
                "scaled_positions": [
                    [0.2210481499999999, 0.1105240749999999, 0.0000000000000000], # 0
                    [0.8894759250000001, 0.1105240749999999, 0.3333333333333333], 
                    [0.8894759250000001, 0.7789518500000001, 0.6666666666666666], 
                    [0.5547867599999998, 0.7773933799999999, 0.0000000000000000], # 3
                    [0.2226066200000001, 0.7773933799999999, 0.3333333333333333], 
                    [0.2226066200000001, 0.4452132400000002, 0.6666666666666666], 
                    [0.8877639499999999, 0.7990755499999997, 0.9248692766666666-1], # 6
                    [0.2009244500000003, 0.0886884000000001, 0.2582026099999999], 
                    [0.8877639499999999, 0.0886884000000001, 0.0751307233333334], # 8
                    [0.9113115999999999, 0.1122360500000001, 0.5915359433333331], 
                    [0.2009244500000003, 0.1122360500000001, 0.7417973900000000], 
                    [0.9113115999999999, 0.7990755499999997, 0.4084640566666666], 
                    [0.2441829899999998, 0.4441736199999998, 0.9246663366666665-1], # 12
                    [0.5558263800000003, 0.8000093700000001, 0.2579996699999998], 
                    [0.2441829899999998, 0.8000093700000001, 0.0753336633333335], # 14
                    [0.1999906299999999, 0.7558170100000001, 0.5913330033333333], 
                    [0.5558263800000003, 0.7558170100000001, 0.7420003300000001], 
                    [0.1999906299999999, 0.4441736199999998, 0.4086669966666667], 
                    [0.5318318299999998, 0.4439072499999996, 0.0756528366666664], # 18
                    [0.5560927500000004, 0.0879245800000002, 0.4089861699999997], 
                    [0.5318318299999998, 0.0879245800000002, 0.9243471633333336-1], # 20
                    [0.9120754199999999, 0.4681681700000002, 0.7423195033333331], 
                    [0.5560927500000004, 0.4681681700000002, 0.5910138300000002], 
                    [0.9120754199999999, 0.4439072499999996, 0.2576804966666669], 
                ],
                "removed_index_1": [0,2,3,5,6,8,9,10,12,14,15,16,18,20,21,22],
                "removed_index_2": [0,3,6,8,12,14,18,20],
            },
            "MX2": {
                "symbols": [self.m_element] * 3 + [self.x_element] * 6,
                "cell": [3.95499, 3.95499, 21.02424, 90, 90, 120],
                "scaled_positions": [
                    [0.0000000000000000, 0.0000000000000000, 0.0000000000000000], # 0
                    [0.6666666666666666, 0.3333333333333333, 0.3333333333333333], 
                    [0.3333333333333333, 0.6666666666666666, 0.6666666666666666], 
                    [0.0000000000000000, 0.0000000000000000, 0.2604615500000000], 
                    [0.6666666666666666, 0.3333333333333333, 0.0728717833333333], # 4
                    [0.6666666666666666, 0.3333333333333333, 0.5937948833333333], 
                    [0.3333333333333333, 0.6666666666666666, 0.4062051166666666], 
                    [0.3333333333333333, 0.6666666666666666, 0.9271282166666667-1], # 7
                    [0.0000000000000000, 0.0000000000000000, 0.7395384500000000], 
                ],
                "removed_index_1": [0,2,4,5,7,8],
                "removed_index_2": [0,4,7],
            },
        }
        return predefined_dict

    def _generate_3d_tmh_structure(self):
        tmh_dict = self.predefined_systems()[self.prototype]
        cell = [
            self.lattice_a if self.lattice_a != 0 else tmh_dict["cell"][0],
            self.lattice_b if self.lattice_b != 0 else tmh_dict["cell"][1],
            self.lattice_c if self.lattice_c != 0 else tmh_dict["cell"][2],
            tmh_dict["cell"][3],
            tmh_dict["cell"][4],
            tmh_dict["cell"][5],
        ]
        return Atoms(symbols=tmh_dict["symbols"], scaled_positions=tmh_dict["scaled_positions"], cell=cell, pbc=True)

    def _generate_single_tmh_structure(self):
        tmh_dict = self.predefined_systems()[self.prototype]
        structure = self._generate_3d_tmh_structure().copy()   
        if self.repeating_unit == "1":
            removed_indices = sorted(tmh_dict["removed_index_1"], reverse=True)
        elif self.repeating_unit == "2":
            removed_indices = sorted(tmh_dict["removed_index_2"], reverse=True)
        elif self.repeating_unit == "3":
            removed_indices = []
        for index in removed_indices:
            del structure[index]
        structure.center()
        return structure

    def build(self):
        structure = self._generate_single_tmh_structure().copy() * self.supercell_matrix

        if self.n_layers > 1:
            single_layer_height = max(atom.position[2] for atom in structure) - min(atom.position[2] for atom in structure)
            for layer in range(1, self.n_layers):
                new_layer = structure.copy()
                for atom in new_layer:
                    atom.position[2] += layer * (single_layer_height + self.layer_distance)
                structure += new_layer
                
        z_length_with_vacuum = (max(atom.position[2] for atom in structure) - min(atom.position[2] for atom in structure)) + self.vacuum
        structure.set_cell([structure.cell[0], structure.cell[1], [0, 0, z_length_with_vacuum]])
        structure.center()
        
        return structure

In [2]:
def identify_system_symmetry(atoms):
    symmetry = get_spacegroup(atoms, symprec=1e-5)
    international_symbol = symmetry.symbol
    spacegroup_number = symmetry.no
    return international_symbol, spacegroup_number
    
system = TrandionMetalHalidesBuilder(prototype="MX3", repeating_unit="3")
atoms = system.build()
print(identify_system_symmetry(atoms))
view(atoms)
# write("mbene.vasp", atoms, direct=True, sort=True, vasp5=True)

('C 2', 5)


<Popen: returncode: None args: ['C:\\Users\\wangchangrui\\.conda\\envs\\wchr...>

### Q-Api 接口

In [ ]:
def trandion_metal_halides_builder(**kwargs):
    kwargs['supercell_matrix'] = (kwargs['n_a'], kwargs['n_b'], 1)
    system = TrandionMetalHalidesBuilder(**kwargs)
    new_structure = system.build()
    cif_output = io.BytesIO()
    write(cif_output, new_structure, format='cif')
    cif_string = cif_output.getvalue().decode('utf-8')
    return cif_string

In [ ]:
@guide_register_func(ModelTag('构建高对称二维过渡金属卤化物结构', nano_type='2d', note='过渡金属卤化物可以分为过渡金属三卤化物 (MX3, X = Cl, Br, I) 和过渡金属二卤化物 (MX2, X = Cl, Br, I) 两个子类。2017 年实验上从块体中成功剥离出单原子层厚的 CrI3, 掀起了二维磁性材料研究的热潮。').identifier)
def TrandionMetalHalides(data):
    mode = data.get('mode')
    if mode == 'init':
        title = Description(name='title', note='构建高对称二维过渡金属卤化物结构，需要指定体相基体元素及晶格参数等信息')
        prototype = SingleFromList(name='prototype', 
                                   note='体相基体类型', 
                                   id='prototype', 
                                   list_value=[{'label': i, 'value': i} for i in ['MX3', 'MX2']], 
                                   default_value='MX3', 
                                   is_required=0)
        m_element = SingleFromList(name='m_element', 
                                   note='体相基体中的 M 元素', 
                                   id='m_element', 
                                   list_value=[{'label': '钪', 'value': 'Sc'},
                                               {'label': '钇', 'value': 'Y' },
                                               {'label': '钆', 'value': 'Gd' },
                                               {'label': '钛', 'value': 'Ti'},
                                               {'label': '锆', 'value': 'Zr'},
                                               {'label': '铪', 'value': 'Hf'},
                                               {'label': '钒', 'value': 'V' },
                                               {'label': '铌', 'value': 'Nb'},
                                               {'label': '钽', 'value': 'Ta'},
                                               {'label': '铬', 'value': 'Cr'},
                                               {'label': '钼', 'value': 'Mo'},
                                               {'label': '钨', 'value': 'W' },
                                               {'label': '锰', 'value': 'Mn' },
                                               {'label': '锝', 'value': 'Tc' },
                                               {'label': '铁', 'value': 'Fe' },
                                               {'label': '钴', 'value': 'Co' },
                                               {'label': '钌', 'value': 'Ru' },
                                               {'label': '铑', 'value': 'Rh' },
                                               {'label': '镍', 'value': 'Ni' }],
                                   default_value='Cr',
                                   is_required=0)
        x_element = SingleFromList(name='x_element', 
                                   note='体相基体中的 X 元素', 
                                   id='x_element', 
                                   list_value=[{'label': '氯', 'value': 'Cl'},
                                               {'label': '溴', 'value': 'Br'},
                                               {'label': '碘', 'value': 'I'}],
                                   default_value='I',
                                   is_required=0)
        repeating_unit = SingleFromList(name='repeating_unit', 
                                        note='体相基体中截取的重复单元', 
                                        id='repeating_unit', 
                                        list_value=[{'label': '1 层', 'value': '1'},
                                                    {'label': '2 层', 'value': '2'},
                                                    {'label': '3 层', 'value': '3'}],
                                   default_value='1',
                                   is_required=0)
        lattice_a = SingleInput(name='lattice_a', note='a 方向晶格常数 (Å)', id='lattice_a', input_type='float', default_value=0.0, min=0.0, max=100.0, is_required=0)
        lattice_b = SingleInput(name='lattice_b', note='b 方向晶格常数 (Å)', id='lattice_b', input_type='float', default_value=0.0, min=0.0, max=100.0, is_required=0)
        lattice_c = SingleInput(name='lattice_c', note='c 方向晶格常数 (Å)', id='lattice_c', input_type='float', default_value=0.0, min=0.0, max=100.0, is_required=0)
        n_a = SingleInput(name='n_a', note='a 轴的单胞重复单元', id='n_a', input_type='int', default_value=3, min=1, max=10, is_required=0)
        n_b = SingleInput(name='n_b', note='b 轴的单胞重复单元', id='n_b', input_type='int', default_value=3, min=1, max=10, is_required=0)
        vacuum = SingleInput(name='vacuum', note='真空层厚度 (Å)', id='vacuum', input_type='float', default_value=15.0, min=10.0, max=100.0, is_required=0)
        n_layers = SingleInput(name='n_layers', note='MBene 重复单元数', id='n_layers', input_type='int', default_value=1, min=1, max=10, is_required=0)
        layer_distance = SingleInput(name='layer_distance', note='MBene 层间距 (Å)', id='layer_distance', input_type='float', default_value=5.0, min=3.0, max=100.0, is_required=0)
        
        return [title(), prototype(), m_element(), x_element(), repeating_unit(), lattice_a(), lattice_b(), lattice_c(), n_a(), n_b(), vacuum(), n_layers(), layer_distance()]
        
    elif mode == 'generate':
        value = data.get('value')
        try:
            cif_string = trandion_metal_halides_builder(**value)
        except Exception as e:
            return {'error':str(e)}
        
        return {'file_content': cif_string, 'file_format': 'cif'}
